In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("data/fm23_preprocessed.csv", sep=",")

In [5]:
df["Position"].unique()

array(['D (C)', 'ST (C)', 'M (L), AM (RLC), ST (C)', 'AM (RLC), ST (C)',
       'AM (L), ST (C)', 'AM (RLC)', 'WB/M/AM (RL)', 'DM, M/AM (C)',
       'M (C)', 'M/AM (LC), ST (C)', 'D (RLC)', 'DM, M (C)',
       'M (LC), AM (RLC)', 'D (LC)', 'M/AM (RLC), ST (C)',
       'AM (LC), ST (C)', 'AM (R), ST (C)', 'D (LC), WB (L)',
       'M (C), AM (LC)', 'D/WB (L)', 'GK', 'D/WB/M (R)',
       'M (R), AM (RC), ST (C)', 'AM (C)', 'M/AM (C)', 'M (LC), AM (C)',
       'D/WB (R)', 'DM, M/AM (C), ST (C)', 'AM (LC)', 'M/AM (RL)',
       'M (C), AM (RLC), ST (C)', 'M/AM (RLC)', 'M (C), AM (RC)',
       'WB (R), M/AM (RC)', 'AM (RL), ST (C)', 'D/WB/M (L)',
       'M/AM (L), ST (C)', 'M (L), AM (RLC)', 'D/WB (R), DM, M (C)',
       'D/WB/M/AM (L)', 'AM (C), ST (C)', 'D (C), DM', 'M/AM (RC)',
       'D/WB (L), DM, M/AM (LC)', 'M/AM (C), ST (C)', 'M (RC), AM (RLC)',
       'M (RC)', 'D/WB/M (R), AM (RL)', 'D (RC)', 'AM (RL)', 'DM, M (RC)',
       'D (RLC), WB (RL)', 'DM, M (RLC)', 'D/WB/M/AM (R)', 'D (R)'

In [6]:
def get_positions(input_string):

    all_positions = []

    for element in input_string.split(','):
        # print("Element", element)
        
        # Step 1: Extract text in parentheses if there is a slash
        try:
            index_of_parentheses = element.index('(')
        except ValueError:
            index_of_parentheses = -1

        if index_of_parentheses >= 0:
            roles = element[:element.index('(')]
            roles = roles.split('/')
            positions = element[element.index('('):].strip("()")
            # print("Position: ", position)
            # extract individual letters from positions (so R, L, C) and save to list as (R), (L), (C)
            positions_list = ['(' + letter + ')' for string in positions for letter in string]

            for role in roles:
                for position in positions_list:
                    all_positions.append(f"{role.strip()} {position}")


        else:
            all_positions.append(f"{element.strip()}")

    # Print the result
    return all_positions

In [7]:
import re

#create list of unique positions
all_positions = []
for i in df["Position"]:
    positions = get_positions(i)
    for pos in positions:
        if pos not in all_positions:
            all_positions.append(pos)
    # #just for debugging
    #     print(pos)
    # print()

all_positions

# Example: 'WB/M/AM (RL)'
# 1. if there is a shlash / -> extract (RL)
# 2. split 'WB/M/AM' by slash
# 3. add '(RL)' for each element

['D (C)',
 'ST (C)',
 'M (L)',
 'AM (R)',
 'AM (L)',
 'AM (C)',
 'WB (R)',
 'WB (L)',
 'M (R)',
 'DM',
 'M (C)',
 'D (R)',
 'D (L)',
 'GK']

In [8]:
# get all positions as list and save them in a list to later replace existing column
positions_total = []
for i in df["Position"]:
    positions = get_positions(i)
    positions_total.append(positions)

# overwrite existing column
df['Position'] = positions_total
df.head()

,UID,Name,DoB,Nat,Division,Club,Preferred Foot,Position,Height,Weight,Age,Transfer Value,Wage,Style
0,831079,Giorgio Chiellini,14/8/1984,ITA,Major League Soccer,LAFC,Left,[D (C)],187,85,37,NaN,90500.0,Leader
1,962988,Gonzalo Higuaín,10/12/1987,ARG,Major League Soccer,Inter Miami,Right,[ST (C)],184,89,34,495000.0,487000.0,Intelligent
2,5132312,Gareth Bale,16/7/1989,WAL,Major League Soccer,LAFC,Left,"[M (L), AM (R), AM (L), AM (C), ST (C)]",185,82,32,3075000.0,135000.0,Creative
3,51000001,Carlos Vela,1/3/1989,MEX,Major League Soccer,LAFC,Left,"[AM (R), AM (L), AM (C), ST (C)]",178,74,33,NaN,340000.0,Creative
4,43043933,Lorenzo Insigne,4/6/1991,ITA,Major League Soccer,Toronto FC,Right,"[AM (L), ST (C)]",163,59,31,NaN,1176000.0,Creative


In [9]:
mapping_positions = {'M (L)':'LM',
                     'M (C)':'CM',
                     'DM':'CDM',
                     'M (R)':'RM',
                     'AM (C)':'CAM',
                     'GK':'GK',
                     'D (R)':'RB',
                     'ST (C)':'ST',
                     'D (C)':'CB',
                     'D (L)':'LB',
                     'AM (R)':'RW',
                     'AM (L)':'LW',
                     'WB (R)':'RWB',
                     'WB (L)':'LWB'}

#map positions (still in list format)
df["Position"] = df["Position"].apply(lambda x: [mapping_positions[pos] for pos in x if pos is not None] if len(x)>0 else [])

#create string attribute only
df["Position_string"] = df["Position"].apply(lambda x: ', '.join(x))

In [10]:
df.head()

,UID,Name,DoB,Nat,Division,Club,Preferred Foot,Position,Height,Weight,Age,Transfer Value,Wage,Style,Position_string
0,831079,Giorgio Chiellini,14/8/1984,ITA,Major League Soccer,LAFC,Left,[CB],187,85,37,NaN,90500.0,Leader,CB
1,962988,Gonzalo Higuaín,10/12/1987,ARG,Major League Soccer,Inter Miami,Right,[ST],184,89,34,495000.0,487000.0,Intelligent,ST
2,5132312,Gareth Bale,16/7/1989,WAL,Major League Soccer,LAFC,Left,"[LM, RW, LW, CAM, ST]",185,82,32,3075000.0,135000.0,Creative,"LM, RW, LW, CAM, ST"
3,51000001,Carlos Vela,1/3/1989,MEX,Major League Soccer,LAFC,Left,"[RW, LW, CAM, ST]",178,74,33,NaN,340000.0,Creative,"RW, LW, CAM, ST"
4,43043933,Lorenzo Insigne,4/6/1991,ITA,Major League Soccer,Toronto FC,Right,"[LW, ST]",163,59,31,NaN,1176000.0,Creative,"LW, ST"


In [12]:
df.to_csv("data/fm23_preprocessed.csv", index=False)